In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

import json

credential = DefaultAzureCredential()

# replace here with your subscription_id and resource_group_name
subscription_id = "???"
resource_group_name = "????"

resource_client = ResourceManagementClient(credential, subscription_id=subscription_id)

resource_type = "Microsoft.CognitiveServices/accounts"  # example resource type
account_kind = 'OpenAI'

print_details = True

resources = list(
    resource_client.resources.list_by_resource_group(
        resource_group_name,
        filter=f"resourceType eq '{resource_type}'"))
# retain  only kind == 'OpenAI'
# resources = [r for r in resources if r.kind == account_kind]

In [ ]:
# define a function to list deployments for a OpenAi endpoint 0
import requests

auth = credential.get_token("https://management.azure.com/.default")
auth_token = auth.token

def send_rest_request(url, method, headers, body):
    headers["Authorization"] = f"Bearer {auth_token}"
    headers["Content-Type"] = "application/json"
    response = requests.request(method, url, headers=headers, data=body)
    if response.status_code == 200:
        return response.json()    
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        requests.raise_for_status(response)

def list_deployments(account_name):
    uri = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}/providers/Microsoft.CognitiveServices/accounts/{account_name}/deployments?api-version=2023-05-01"
    response = send_rest_request(uri, "GET", {}, "")
    return response



In [ ]:
for r in resources:
    resource = resource_client.resources.get_by_id(r.id, "2021-04-30")  
    print("=====================================")
    print(resource.name)
    provisioned = resource.properties['provisioningState'] == 'Succeeded'   
    print(f"provisioning state: {resource.properties['provisioningState']}")
    if print_details: 
        print(f"""
# +++++++++++++++++++++++++
# {json.dumps(resource.serialize(), indent=2)}
# ------------------------""")
    if (provisioned):
        print(f"OAI Studio URLs: ")
        print(f" deployments URL - https://oai.azure.com/portal/{resource.properties['internalId']}/deployment")
        print(f" quota (at subscription level) - https://oai.azure.com/portal/{resource.properties['internalId']}/quota")
        print("~~~~~~~~ BEGIN deployments:")
        print(json.dumps(list_deployments(resource.name), indent=2))
        print("~~~~~~~~ END deployments.")
    else:
        print(f"OAI Studio URLs: will only work if provisioning state is Succeeded. Skipping.")
    
# %%